In [4]:
import pandas as pd
import numpy as np
import boto3

In [ ]:
import sys
sys.path.append("/Users/gabriel/Documents/Git/End-to-end MLOps for Time Series")
from utils import load_config
config = load_config("../config/config.yaml")

In [ ]:
# Read data
#TODO: set 'target' as the first column. Try this:
# reordered_columns = pd.Index(["target"]).append(data.columns.drop("target"))
# data = data[reordered_columns]
data = pd.read_csv("../data/raw/train.csv")
print("data shape :", data.shape)
data.head(5)

data shape : (2018352, 9)


,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2


In [7]:
# Seperate consumption and production data
consumption = data[data["is_consumption"]==1]
print("consumption data shape :", consumption.shape)
display(consumption.head())
production = data[data["is_consumption"]==0]
print("production data shape :", production.shape)
display(production.head())

consumption data shape : (1009176, 9)


,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1
5,0,0,3,656.859,1,2021-09-01 00:00:00,0,5,2
7,0,1,0,59.000,1,2021-09-01 00:00:00,0,7,3
9,0,1,1,501.760,1,2021-09-01 00:00:00,0,9,4


production data shape : (1009176, 9)


,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2
6,0,1,0,0.000,0,2021-09-01 00:00:00,0,6,3
8,0,1,1,0.000,0,2021-09-01 00:00:00,0,8,4


In [8]:
len(data["data_block_id"].unique())

638

In [9]:
# consumption.set_index("datetime", inplace=True)
# consumption.index
# consumption.index = pd.to_datetime(consumption.index)

# Missing datetimes

## Consumption

In [10]:
# at each time step, a new value is generated per 'prediction_unit_id'
consumption = consumption.sort_values(by=["prediction_unit_id", "datetime"])
consumption.head()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0
123,0,0,1,77.691,1,2021-09-01 01:00:00,0,123,0
245,0,0,1,91.594,1,2021-09-01 02:00:00,0,245,0
367,0,0,1,87.955,1,2021-09-01 03:00:00,0,367,0
489,0,0,1,88.184,1,2021-09-01 04:00:00,0,489,0


In [11]:
consumption.insert(loc=6, column="datetime-1", value=consumption["datetime"].shift(1))
consumption.head()

,county,is_business,product_type,target,is_consumption,datetime,datetime-1,data_block_id,row_id,prediction_unit_id
1,0,0,1,96.590,1,2021-09-01 00:00:00,None,0,1,0
123,0,0,1,77.691,1,2021-09-01 01:00:00,2021-09-01 00:00:00,0,123,0
245,0,0,1,91.594,1,2021-09-01 02:00:00,2021-09-01 01:00:00,0,245,0
367,0,0,1,87.955,1,2021-09-01 03:00:00,2021-09-01 02:00:00,0,367,0
489,0,0,1,88.184,1,2021-09-01 04:00:00,2021-09-01 03:00:00,0,489,0


In [12]:
consumption["datetime"] = pd.to_datetime(consumption["datetime"])
consumption["datetime-1"] = pd.to_datetime(consumption["datetime-1"])

In [13]:
consumption.insert(loc=7, column="timestep", value=consumption["datetime"]-consumption["datetime-1"])
consumption.head()

,county,is_business,product_type,target,is_consumption,datetime,datetime-1,timestep,data_block_id,row_id,prediction_unit_id
1,0,0,1,96.590,1,2021-09-01 00:00:00,NaT,NaT,0,1,0
123,0,0,1,77.691,1,2021-09-01 01:00:00,2021-09-01 00:00:00,0 days 01:00:00,0,123,0
245,0,0,1,91.594,1,2021-09-01 02:00:00,2021-09-01 01:00:00,0 days 01:00:00,0,245,0
367,0,0,1,87.955,1,2021-09-01 03:00:00,2021-09-01 02:00:00,0 days 01:00:00,0,367,0
489,0,0,1,88.184,1,2021-09-01 04:00:00,2021-09-01 03:00:00,0 days 01:00:00,0,489,0


In [14]:
max_timestep_per_unit = consumption[["prediction_unit_id", "timestep"]].groupby(by="prediction_unit_id").max()
# identify discontinuous series, i.e., those which contain a timestep different from 1 hour
discontinuous_series = max_timestep_per_unit[max_timestep_per_unit["timestep"]!=pd.Timedelta(1, "h")].index
max_timestep_per_unit.loc[discontinuous_series]

,timestep
prediction_unit_id,
21,31 days 01:00:00
26,62 days 01:00:00
41,173 days 01:00:00
44,31 days 01:00:00
47,63 days 01:00:00
68,76 days 01:00:00


In [15]:
consumption = consumption[~consumption["prediction_unit_id"].isin(discontinuous_series)]

In [16]:
consumption["prediction_unit_id"].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 42, 43, 45, 46, 48, 49, 50, 51, 52, 53, 54, 55,
       56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67])

## Production

In [17]:
# at each time step, a new value is generated per 'prediction_unit_id'
production = production.sort_values(by=["prediction_unit_id", "datetime"])
production.head()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0
122,0,0,1,1.132,0,2021-09-01 01:00:00,0,122,0
244,0,0,1,0.490,0,2021-09-01 02:00:00,0,244,0
366,0,0,1,0.496,0,2021-09-01 03:00:00,0,366,0
488,0,0,1,0.149,0,2021-09-01 04:00:00,0,488,0


In [18]:
production.insert(loc=6, column="datetime-1", value=production["datetime"].shift(1))
production.head()

,county,is_business,product_type,target,is_consumption,datetime,datetime-1,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,None,0,0,0
122,0,0,1,1.132,0,2021-09-01 01:00:00,2021-09-01 00:00:00,0,122,0
244,0,0,1,0.490,0,2021-09-01 02:00:00,2021-09-01 01:00:00,0,244,0
366,0,0,1,0.496,0,2021-09-01 03:00:00,2021-09-01 02:00:00,0,366,0
488,0,0,1,0.149,0,2021-09-01 04:00:00,2021-09-01 03:00:00,0,488,0


In [19]:
production["datetime"] = pd.to_datetime(production["datetime"])
production["datetime-1"] = pd.to_datetime(production["datetime-1"])

In [20]:
production.insert(loc=7, column="timestep", value=production["datetime"]-production["datetime-1"])
production.head()

,county,is_business,product_type,target,is_consumption,datetime,datetime-1,timestep,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,NaT,NaT,0,0,0
122,0,0,1,1.132,0,2021-09-01 01:00:00,2021-09-01 00:00:00,0 days 01:00:00,0,122,0
244,0,0,1,0.490,0,2021-09-01 02:00:00,2021-09-01 01:00:00,0 days 01:00:00,0,244,0
366,0,0,1,0.496,0,2021-09-01 03:00:00,2021-09-01 02:00:00,0 days 01:00:00,0,366,0
488,0,0,1,0.149,0,2021-09-01 04:00:00,2021-09-01 03:00:00,0 days 01:00:00,0,488,0


In [21]:
max_timestep_per_unit = production[["prediction_unit_id", "timestep"]].groupby(by="prediction_unit_id").max()
# identify discontinuous series, i.e., those which contain a timestep different from 1 hour
discontinuous_series = max_timestep_per_unit[max_timestep_per_unit["timestep"]!=pd.Timedelta(1, "h")].index
max_timestep_per_unit.loc[discontinuous_series]

,timestep
prediction_unit_id,
21,31 days 01:00:00
26,62 days 01:00:00
41,173 days 01:00:00
44,31 days 01:00:00
47,63 days 01:00:00
68,76 days 01:00:00


In [22]:
production = production[~production["prediction_unit_id"].isin(discontinuous_series)]

In [23]:
production["prediction_unit_id"].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 42, 43, 45, 46, 48, 49, 50, 51, 52, 53, 54, 55,
       56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67])

In [24]:
consumption["prediction_unit_id"].unique().all() == production["prediction_unit_id"].unique().all()

np.True_

Logically, the discontinuities were located at the same time both for consumption and production.

In [25]:
consumption.drop(columns=["datetime-1", "timestep"], inplace=True)
production.drop(columns=["datetime-1", "timestep"], inplace=True)

# Missing values

## Consumption

In [26]:
consumption.isna().sum()

county                  0
is_business             0
product_type            0
target                248
is_consumption          0
datetime                0
data_block_id           0
row_id                  0
prediction_unit_id      0
dtype: int64

In [27]:
consumption[consumption["target"].isna()].head()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
178939,0,0,1,NaN,1,2021-10-31 03:00:00,60,178939,0
634867,0,0,1,NaN,1,2022-03-27 03:00:00,207,634867,0
1332457,0,0,1,NaN,1,2022-10-30 03:00:00,424,1332457,0
1806253,0,0,1,NaN,1,2023-03-26 03:00:00,571,1806253,0
178941,0,0,2,NaN,1,2021-10-31 03:00:00,60,178941,1


In [28]:
mask = ((consumption["datetime"] <= "2021-10-31 04:00:00")
        & (consumption["datetime"] >= "2021-10-31 02:00:00")
        & (consumption["prediction_unit_id"] == 0))
consumption[mask]

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
178813,0,0,1,166.141,1,2021-10-31 02:00:00,60,178813,0
178939,0,0,1,NaN,1,2021-10-31 03:00:00,60,178939,0
179065,0,0,1,174.856,1,2021-10-31 04:00:00,60,179065,0


In [29]:
consumption.ffill(inplace=True)
consumption.isna().sum()

county                0
is_business           0
product_type          0
target                0
is_consumption        0
datetime              0
data_block_id         0
row_id                0
prediction_unit_id    0
dtype: int64

In [30]:
consumption[mask]

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
178813,0,0,1,166.141,1,2021-10-31 02:00:00,60,178813,0
178939,0,0,1,166.141,1,2021-10-31 03:00:00,60,178939,0
179065,0,0,1,174.856,1,2021-10-31 04:00:00,60,179065,0


## Production

In [31]:
production.isna().sum()

county                  0
is_business             0
product_type            0
target                248
is_consumption          0
datetime                0
data_block_id           0
row_id                  0
prediction_unit_id      0
dtype: int64

In [32]:
production[production["target"].isna()].head()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
178938,0,0,1,NaN,0,2021-10-31 03:00:00,60,178938,0
634866,0,0,1,NaN,0,2022-03-27 03:00:00,207,634866,0
1332456,0,0,1,NaN,0,2022-10-30 03:00:00,424,1332456,0
1806252,0,0,1,NaN,0,2023-03-26 03:00:00,571,1806252,0
178940,0,0,2,NaN,0,2021-10-31 03:00:00,60,178940,1


In [33]:
mask = ((production["datetime"] <= "2021-10-31 04:00:00")
        & (production["datetime"] >= "2021-10-31 02:00:00")
        & (production["prediction_unit_id"] == 0))
production[mask]

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
178812,0,0,1,0.0,0,2021-10-31 02:00:00,60,178812,0
178938,0,0,1,NaN,0,2021-10-31 03:00:00,60,178938,0
179064,0,0,1,0.0,0,2021-10-31 04:00:00,60,179064,0


In [34]:
production.ffill(inplace=True)
production.isna().sum()

county                0
is_business           0
product_type          0
target                0
is_consumption        0
datetime              0
data_block_id         0
row_id                0
prediction_unit_id    0
dtype: int64

In [35]:
production[mask]

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
178812,0,0,1,0.0,0,2021-10-31 02:00:00,60,178812,0
178938,0,0,1,0.0,0,2021-10-31 03:00:00,60,178938,0
179064,0,0,1,0.0,0,2021-10-31 04:00:00,60,179064,0


# Downcasting

In [36]:
df = pd.concat([consumption, production])
df

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0
123,0,0,1,77.691,1,2021-09-01 01:00:00,0,123,0
245,0,0,1,91.594,1,2021-09-01 02:00:00,0,245,0
367,0,0,1,87.955,1,2021-09-01 03:00:00,0,367,0
489,0,0,1,88.184,1,2021-09-01 04:00:00,0,489,0
...,...,...,...,...,...,...,...,...,...
2017796,11,1,0,6.320,0,2023-05-31 19:00:00,637,2017796,67
2017926,11,1,0,2.633,0,2023-05-31 20:00:00,637,2017926,67
2018056,11,1,0,0.122,0,2023-05-31 21:00:00,637,2018056,67
2018186,11,1,0,0.000,0,2023-05-31 22:00:00,637,2018186,67


In [37]:
df.dtypes

county                         int64
is_business                    int64
product_type                   int64
target                       float64
is_consumption                 int64
datetime              datetime64[ns]
data_block_id                  int64
row_id                         int64
prediction_unit_id             int64
dtype: object

In [38]:
int_columns = list(df.dtypes[df.dtypes == np.int64].index)
float_columns = list(df.dtypes[df.dtypes == np.float64].index)

In [39]:
int_columns

['county',
 'is_business',
 'product_type',
 'is_consumption',
 'data_block_id',
 'row_id',
 'prediction_unit_id']

In [40]:
float_columns

['target']

In [41]:
df.min()

county                                  0
is_business                             0
product_type                            0
target                                0.0
is_consumption                          0
datetime              2021-09-01 00:00:00
data_block_id                           0
row_id                                  0
prediction_unit_id                      0
dtype: object

In [42]:
for col in int_columns:
    df[col] = pd.to_numeric(df[col], downcast="unsigned")

In [43]:
for col in float_columns:
    df[col] = pd.to_numeric(df[col], downcast="float")

In [44]:
df.dtypes

county                         uint8
is_business                    uint8
product_type                   uint8
target                       float32
is_consumption                 uint8
datetime              datetime64[ns]
data_block_id                 uint16
row_id                        uint32
prediction_unit_id             uint8
dtype: object

# Train-Test split

In [46]:
df = df.sort_values(by="datetime")
df

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
1,0,0,1,96.589996,1,2021-09-01 00:00:00,0,1,0
25,3,0,3,39.241001,1,2021-09-01 00:00:00,0,25,12
66,8,1,3,0.000000,0,2021-09-01 00:00:00,0,66,33
106,14,0,1,0.000000,0,2021-09-01 00:00:00,0,106,53
64,8,0,3,0.000000,0,2021-09-01 00:00:00,0,64,32
...,...,...,...,...,...,...,...,...,...
2018284,7,1,0,0.000000,0,2023-05-31 23:00:00,637,2018284,28
2018286,7,1,1,0.000000,0,2023-05-31 23:00:00,637,2018286,29
2018288,7,1,3,1.450000,0,2023-05-31 23:00:00,637,2018288,30
2018258,4,0,1,0.002000,0,2023-05-31 23:00:00,637,2018258,15


In [47]:
df.iloc[int(len(df) * 0.80)]

county                                  5
is_business                             1
product_type                            3
target                        1646.064941
is_consumption                          1
datetime              2023-01-26 05:00:00
data_block_id                         512
row_id                            1618363
prediction_unit_id                     23
Name: 1618363, dtype: object

In [48]:
train = df[df["datetime"] <= "2023-01-26 05:00:00"]
test = df[df["datetime"] > "2023-01-26 05:00:00"]
print(train.shape)
print(test.shape)

(1521300, 9)
(380268, 9)


In [49]:
len(df) == len(train) + len(test)

True

In [50]:
len(train) / len(df)

0.8000239802100162

In [51]:
len(test) / len(df)

0.19997601978998383

In [55]:
production_train = train[train["is_consumption"] == 0]
production_test = test[test["is_consumption"] == 0]

In [56]:
consumption_train = train[train["is_consumption"] == 1]
consumption_test = test[test["is_consumption"] == 1]

# Save cleaned data

In [62]:
import os

In [63]:
if "preprocessed" not in os.listdir("../data"):
    os.mkdir("../data/preprocessed")

In [ ]:
# TODO: save in an other format to preserve data types
train.to_csv("../data/preprocessed/train.csv", index=False)
production_train.to_csv("../data/preprocessed/production_train.csv", index=False)
consumption_train.to_csv("../data/preprocessed/consumption_train.csv", index=False)

In [60]:
test.to_csv("../data/preprocessed/test.csv", index=False)
production_test.to_csv("../data/preprocessed/production_test.csv", index=False)
consumption_test.to_csv("../data/preprocessed/consumption_test.csv", index=False)

In [64]:
os.listdir("../data/preprocessed")

['consumption_train.csv',
 'test.csv',
 'consumption_test.csv',
 'production_train.csv',
 'train.csv',
 'production_test.csv']

In [ ]:
# Load data to AWS S3 (Optional)
if config["s3_bucket"]:
    s3_client = boto3.client('s3')
    with open("../data/preprocessed/train.csv", "rb") as file:
        s3_client.upload_fileobj(file, config["s3_bucket"], "data/preprocessed/train.csv")
    with open("../data/preprocessed/test.csv", "rb") as file:
        s3_client.upload_fileobj(file, config["s3_bucket"], "data/preprocessed/test.csv")

In [ ]:
# Check that data has been uploaded correctly
if config["s3_bucket"]:
    response = s3_client.list_objects(
        Bucket=config["s3_bucket"])
    for obj in response.get("Contents"):
        print(obj.get("Key"))